In [1]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [2]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
! nvidia-smi

Thu Jul  7 16:26:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 67%   73C    P2    83W / 260W |   2071MiB / 11019MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 31%   

# MongoDB

In [4]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    db_de = db.patent_de
    db_us = db.patent_us
    db_cn = db.patent_cn
    # db_test1 = db.test_1
    # db_test2 = db.test_2
    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de
    db_rm_cn_5 = db.rm_patent_cn_5
    db_rm_cn_10 = db.rm_patent_cn_10
    db_rm_us_5 = db.rm_patent_us_5
    db_rm_us_10 = db.rm_patent_us_10
    db_rm_de_5 = db.rm_patent_de_5
    db_rm_de_10 = db.rm_patent_de_10
except errors.ConnectionFailure as err:
    print(err)

In [6]:
db_rm_de

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'rm_patent_de')

## DE

In [50]:
data_assignee = db_de.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [51]:
len(ls_company)

446

In [7]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_de.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc_de = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc_de

,company,cpc,count
0,Intel Corp,G06F9,392
1,Intel Corp,H01L2224,239
2,Intel Corp,H04L67,157
3,Intel Corp,G06N3,153
4,Intel Corp,G06F12,134
...,...,...,...
14385,Trumpf Werkzeugmaschinen GmbH and Co KG,00,1
14386,Trumpf Werkzeugmaschinen GmbH and Co KG,1474,1
14387,Trumpf Werkzeugmaschinen GmbH and Co KG,H05K7,1
14388,Trumpf Werkzeugmaschinen GmbH and Co KG,043,1


In [8]:
df_cpc_de.shape

(14390, 3)

In [9]:
# 取cpc前3碼
df_cpc_de["cpc_n"] = df_cpc_de.cpc.apply(lambda x: x[0:3])

In [10]:
df_cpc_de

,company,cpc,count,cpc_n
0,Intel Corp,G06F9,392,G06
1,Intel Corp,H01L2224,239,H01
2,Intel Corp,H04L67,157,H04
3,Intel Corp,G06N3,153,G06
4,Intel Corp,G06F12,134,G06
...,...,...,...,...
14385,Trumpf Werkzeugmaschinen GmbH and Co KG,00,1,00
14386,Trumpf Werkzeugmaschinen GmbH and Co KG,1474,1,147
14387,Trumpf Werkzeugmaschinen GmbH and Co KG,H05K7,1,H05
14388,Trumpf Werkzeugmaschinen GmbH and Co KG,043,1,043


In [11]:
# 轉成company-cpc 的總數表格
table_de = df_cpc_de.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [12]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,A01,A24,A41,A42,A43,A44,A45,A47,A61,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
1,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
2,ALBA GROUP PLC & Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,AT&S Austria Technologie and Systemtechnik AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0
4,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
final_table_de.shape

(437, 84)

In [67]:
final_table_de.to_csv('company_cpc_de.csv',index=0)

In [14]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.company_cpc_de

In [15]:
# title
conn_db_collection.insert_one({"company":final_table_de.columns.tolist()[0],"cpc":final_table_de.columns.tolist()[1:]})

In [16]:
# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

## CN

In [10]:
data_assignee = db_cn.aggregate([
  {"$lookup": {
      "from": "db_us",
      "localField": "priority",
      "foreignField": "patentNumber",
      "as": "test"}},
  {"$match": {
      "test.patentNumber": {
        "$exists": False}
    }},
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
    
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [24]:
data_assignee = db_cn.aggregate([
    {"$lookup": {
      "from": "db_us",
      "localField": "priority",
      "foreignField": "patentNumber",
      "as": "test"}},
    {"$match": {
      "test.patentNumber": {
        "$exists": False}
    }},
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [25]:
len(ls_company)

13002

In [28]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_cn_rm.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

AttributeError: 'CommandCursor' object has no attribute 'aggregate'

In [20]:
df_cpc.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,5583
1,Qualcomm Inc,H04W72,4430
2,Qualcomm Inc,H04L1,2496
3,Qualcomm Inc,H04B7,2182
4,Qualcomm Inc,H04W52,1185


In [19]:
df_cpc.shape

(177888, 3)

In [21]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [22]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [23]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A21,A23,A24,A41,A42,A43,A44,A45,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,(c-Dot),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
1,01 Bulletin Laboratory Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
2,10757616 Canada Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,28.0,0.0,0.0,0.0,2.0,2.0,0.0
3,10757616加拿大公司,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,12.0,28.0,0.0,0.0,0.0,2.0,2.0,0.0
4,"11th Street Co., Ltd.",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,1.0,0.0,0.0


In [24]:
final_table_cn.shape

(8608, 124)

In [66]:
final_table_cn.to_csv('company_cpc_cn_rm.csv',index=0)

In [25]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.company_cpc_cn

In [26]:
# title
conn_db_collection.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

In [27]:
# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

## US

In [54]:
data_assignee = db_us.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [55]:
len(ls_company)

5623

In [29]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    try:
        data_cpc = db_us.aggregate([
            {"$match" : {"current_assignee":{"$regex":c}}},
            {"$unwind" : "$current_assignee"},
            {"$unwind" : "$classification"},
            {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
            {"$unwind" : "$class"},
            {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
            {"$sort": {"count":-1}}
            # {"$limit":5}
        ])
    except:
        print(c)

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc_us = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc_us.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,10569
1,Qualcomm Inc,H04W72,9022
2,Qualcomm Inc,H04L1,4813
3,Qualcomm Inc,H04B7,4541
4,Qualcomm Inc,H04W52,2422


In [30]:
df_cpc_us.shape

(225532, 3)

In [31]:
# 取cpc前3碼
df_cpc_us["cpc_n"] = df_cpc_us.cpc.apply(lambda x: x[0:3])

In [32]:
# 轉成company-cpc 的總數表格
table_us = df_cpc_us.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [34]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01 COMMUNIQUE LABORATORY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
2,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
3,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
4,10757616 Canada Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,14.0,0.0,0.0,0.0,1.0,1.0,0.0


In [35]:
final_table_us.shape

(5587, 121)

In [65]:
final_table_us.to_csv('company_cpc_us_rm.csv',index=0)

In [39]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.company_cpc_us

In [40]:
# title
conn_db_collection.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

In [41]:
# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

## 合併

In [59]:
# concat de cn
df_c = pd.concat([final_table_de,final_table_cn], join='outer')
df_c2 = pd.concat([df_c,final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
1,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
2,ALBA GROUP PLC & Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,AT&S Austria Technologie and Systemtechnik AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0
4,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5582,incNETWORKS Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0
5583,mSignia Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
5584,nChain Holdings Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0
5585,u-blox AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,27.0,0.0,1.0,0.0,0.0


In [60]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,(c-Dot),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,0.0
2,01 Bulletin Laboratory Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
3,01 COMMUNIQUE LABORATORY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
4,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13364,incNETWORKS Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0
13365,mSignia Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
13366,nChain Holdings Ltd,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,138.0,0.0,0.0,0.0,0.0
13367,u-blox AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,84.0,0.0,2.0,0.0,0.0


In [61]:
df_c2.to_csv('company_cpc_all_rm.csv',index=0)

In [62]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.company_cpc_all

In [63]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

In [64]:
# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

## Test

In [19]:
test1 = pd.DataFrame(list(db_test1.find()))

In [20]:
test1.head()

,_id,appDate,priority
0,626d580bb9455959c69fc4b3,20100501,1
1,626d580bb9455959c69fc4b4,20100701,2
2,626d580bb9455959c69fc4b5,20100801,3
3,6272409c5477582d912a27fb,20100901,3


In [17]:
test2 = pd.DataFrame(db_test2.aggregate([{"$project":{"priority":1}}]))

In [18]:
test2

,_id,priority
0,626d576432510447df0abe89,1
1,626d576432510447df0abe8a,21
2,626d576432510447df0abe8b,3


In [21]:
a = test1.append(test2)

/tmp/ipykernel_31739/1407032937.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  a = test1.append(test2)


In [22]:
a

,_id,appDate,priority
0,626d580bb9455959c69fc4b3,20100501,1
1,626d580bb9455959c69fc4b4,20100701,2
2,626d580bb9455959c69fc4b5,20100801,3
3,6272409c5477582d912a27fb,20100901,3
0,626d576432510447df0abe89,NaN,1
1,626d576432510447df0abe8a,NaN,21
2,626d576432510447df0abe8b,NaN,3


In [23]:
result = a.drop_duplicates(subset=['priority'],keep=False)

In [24]:
result

,_id,appDate,priority
1,626d580bb9455959c69fc4b4,20100701,2
1,626d576432510447df0abe8a,NaN,21


In [27]:
r = pd.concat([test1,test2,test2]).drop_duplicates(subset=['priority'],keep=False)

In [28]:
r

,_id,appDate,priority
1,626d580bb9455959c69fc4b4,20100701,2


# 前五年

## CN-US

### 去除CN中priority和US重複的

In [29]:
dbcn = pd.DataFrame(list(db_cn.find()))

In [30]:
dbcn.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,61fd2531f4d57576a5021667,CN103890730B,"[普拉蒂普·米斯拉, 阿尔潘·帕尔, 巴拉穆拉利达尔·普鲁肖撒曼, 集拉布拉塔·包米克, 迪...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20170711,N/A,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",本文公开了用于车辆遥测应用的智能开发、部署和管理的计算平台。此外，本发明提供了一种允许在基于...,用于传感器驱动的车辆遥测应用和服务的开发和部署的计算平台,用于传感器驱动的车辆遥测应用和服务的开发和部署的计算 平台技术领域[0001]本发明通常涉及...,[1.一种智能交通系统，包括；(a)基础设施平台，其包括多个硬件和软件组件、基础设施服务、应...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'CN', 'patentNumber': '163364', 'kind...","[{'pto': 'US', 'patentNumber': '527662', 'kind...",None,"[{'pto': 'US', 'patentNumber': '9932033', 'kin...","[{'publication': 'CN103890730B', 'time': '2017...",IN2651/MUM/2011
1,61fd25e1f4d57576a5021669,CN103492389B,"[M·阿尔姆施泰特尔, M·托尔曼, A·特雷姆尔, N·特劳贝]",[Origenis GmbH],[Origenis GmbH],20120423,20160914,N/A,"[C07D487/04, A61K31/519, A61P1/00, A61P11/00, ...",本发明涉及式(I)的新化合物，其能够抑制一种或多种激酶，尤其抑制SYK(脾酪氨酸激酶)、LR...,"用作激酶抑制剂的吡唑并[4,3-d]嘧啶",技术领域[0001 ]本发明涉及能够抑制一种或多种激酶，尤其SYK(脾酪氨酸激酶）、LRRK...,[1.式(I)的化合物或其药学上可接受的盐其中A为NH;Ri为任选地被取代的杂烷基、C6-1...,"[US, EP, JP, KR, CN, AP, AU, BR, CA, CL, CO, C...","[{'pto': 'WO', 'patentNumber': '1998029413', '...","[{'pto': 'US', 'patentNumber': '446986', 'kind...",None,"[{'pto': 'KR', 'patentNumber': '2014002097', '...","[{'publication': 'CN103492389B', 'time': '2016...",US201161517582P
2,61fd25e5f4d57576a502166b,CN102393961A,"[王滨海, 张晶晶, 王骞, 刘俍, 王万国]",[State Grid Intelligent Technology Co Ltd],[Electric Power Research Institute of State Gr...,20110629,20120328,N/A,None,本发明公开了一种在复杂自然环境背景下，实现了基于计算机视觉的飞行器输电线路实时检测跟踪方法。...,基于计算机视觉的飞行器输电线路实时检测跟踪方法,基于计算机视觉的飞行器输电线路实时检测跟踪方法技术领域[0001]\t本发明涉及一种图像处理...,[1.一种基于计算机视觉的飞行器输电线路实时检测跟踪方法，其特征是，首先使用抗噪声的梯度图像...,[CN],None,None,"[{'pto': 'CN', 'patentNumber': '10314993', 'ki...",None,"[{'publication': 'CN102393961A', 'time': '2012...",CN201110179733.2A
3,61fd26a7f4d57576a502166d,CN103155174B,"[迈克尔·尤金·杨, 阿琼·丹尼尔·瑞尼瓦斯, 马修·R.·鲁滨逊, 亚历山大·周·米塔尔]",[Chen Hung Technology Holdings Limited],[Chen Hung Technology Holdings Ltd],20110808,20170623,N/A,"[H05K1/09, H01L31/048, G06F3/0445, H01L31/0481...",将活性或功能性添加剂嵌入到用作包括太阳能装置、智能窗、显示器等的各种电子或光电装置的组件的主...,具有表面嵌入的添加剂的装置组件和相关的制造方法,具有表面嵌入的添加剂的装置组件和相关的制造方法[0001] 相关申请的交叉引用[0002] ...,[1.一种太阳能装置，包括：前覆盖层；后覆盖层；光敏层，所述光敏层被设置在所述前覆盖层与所述...,"[US, EP, JP, KR, CN, AU, CA, WO]","[{'pto': 'CN', 'patentNumber': '10108814', 'ki...","[{'pto': 'US', 'patentNumber': '375128', 'kind...",None,"[{'pto': 'US', 'patentNumber': '9530940', 'kin...","[{'publication': 'CN103155174B', 'time': '2017...",US37168810P
4,61fd26c2f4d57576a502166f,CN103958612B,"[服部公彦, 玉井晃义, 歌崎宪, 歌崎宪一]",[Toray Industries Inc],[Toray Industries Inc],20121119,20160824,N/A,"[C08K3/04, C08L101/00, B29B7/90, B29B9/06, B29...",本发明涉及一种碳纤维增强热塑性树脂组合物及其成型品，所述成型品为使用含有(A)热塑性树脂、(...,碳纤维增强热塑性树脂组合物、该组合物的粒料和成型品,技术领域[0001] 本发明的课题在于提供一种机械特性、表面外观等优异、特别是弯曲弹性模量、...,[1.一种碳纤维增强热塑性树脂组合物，为含有(A)热塑性树脂、（B)碳纤维和(C)钛化合物的...,"[US, EP, JP, KR, CN, TW, WO]","[{'pto': 'CN', 'patentNumber': '10179613', 'ki...","[{'pto': 'JP', 'patentNumber': 'S629541', 'kin...",None,"[{'pto': 'JP', 'patentNumber': '5987335', 'kin...","[{'publication': 'CN103958612B', 'time': '2016...",JP2011-259853


In [31]:
dbus = pd.DataFrame(list(db_us.find()))

In [32]:
dbus.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,62009a5e7b73c16636b31082,US9990182B2,"[Prateep Misra, Arpan Pal, Balamuralidhar Puru...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20180605,20180516,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",A computing platform for intelligent developme...,Computerplatformfordevelopmentanddeploymentofs...,The present application claims the benefit of ...,[1.Anintelligenttransportationsystemcomprising...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'US', 'patentNumber': '527662', 'kind...","[{'pto': 'JP', 'patentNumber': '200126069', 'k...","[{'pto': 'US', 'patentNumber': '10965787', 'ki...","[{'pto': 'US', 'patentNumber': '9932033', 'kin...","[{'publication': 'US9990182B2', 'time': '2018-...",IN2651/MUM/2011
1,62009a807b73c16636b31084,US20190027414A1,"[Senthil Kumar Ramadas, Saravanan Shanmugavel]","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...",20180924,20190124,N/A,"[B32B7/02, H01L23/29, B82Y30/00, C23C28/00, C2...","Disclosed is an encapsulation barrier stack, c...",Encapsulationbarrierstack,The present application is a divisional applic...,[1.Amethodofmanufacturinganencapsulationbarrie...,"[US, EP, JP, KR, CN, AU, CA, DK, MX, MY, RU, S...","[{'pto': 'US', 'patentNumber': '20100089636', ...","[{'pto': 'US', 'patentNumber': '560778', 'kind...","[{'pto': 'US', 'patentNumber': '10854853', 'ki...","[{'pto': 'US', 'patentNumber': '20150314941', ...","[{'publication': 'US20190027414A1', 'time': '2...",US201161550764P
2,62009aea7b73c16636b31086,US10216485B2,"[Prateep Misra, Arpan Pal, Balamuralidhar Puru...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20190226,20190206,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",A method and system for real-time analytics of...,Computerplatformfordevelopmentanddeploymentofs...,The present application claims the benefit of ...,[1.Anintelligenttransportationsystemcomprising...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'US', 'patentNumber': '20020128751', ...","[{'pto': 'US', 'patentNumber': '7779383', 'kin...","[{'pto': 'US', 'patentNumber': '20180373418', ...","[{'pto': 'US', 'patentNumber': '8250213', 'kin...","[{'publication': 'US10216485B2', 'time': '2019...",IN2651/MUM/2011
3,62009b557b73c16636b31088,US20140310243A1,"[Steven James McGee, Tammy Lee McGee]",[],[Mr. Steven James McGee],20120813,20141016,N/A,"[G06F17/30575, G06F16/27, G06Q10/101, G06Q10/1...","Systemic, adaptive procedural template compris...",Heartbeaconcycle,Related applications by Applicant Steven J. Mc...,"[1.Asystemic,adaptiveproceduraltemplatemethodu...",[US],"[{'pto': 'US', 'patentNumber': '20060217138', ...",None,"[{'pto': 'US', 'patentNumber': '20130086129', ...",None,"[{'publication': 'US20140310243A1', 'time': '2...","US12/856,715"
4,62009b717b73c16636b3108a,US8965375B2,"[Juergen Michel, Dirk Rose]",[Nokia Solutions and Networks Oy],[Nokia Solutions and Networks Oy],20110504,20150224,20150204,"[H04W36/04, H04W24/10, H04W52/0206, H04W52/023...",There are provided measures for a pathloss-bas...,Pathlossbasedaccessnodewake-upcontrol,The present invention relates to a pathloss-ba...,[1.Amethodcomprisingretrievinganaveragehandove...,"[US, EP, WO]","[{'pto': 'US', 'patentNumber': '6498932', 'kin...",None,"[{'pto': 'US', 'patentNumber': '20150043538', ...","[{'pto': 'US', 'patentNumber': '9203489', 'kin...","[{'publication': 'US8965375B2', 'time': '2015-...",PCT/EP2011/057133


In [33]:
result = pd.concat([dbcn,dbus,dbus]).drop_duplicates(subset=['priority'],keep=False)

In [34]:
result.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
1,61fd25e1f4d57576a5021669,CN103492389B,"[M·阿尔姆施泰特尔, M·托尔曼, A·特雷姆尔, N·特劳贝]",[Origenis GmbH],[Origenis GmbH],20120423,20160914,N/A,"[C07D487/04, A61K31/519, A61P1/00, A61P11/00, ...",本发明涉及式(I)的新化合物，其能够抑制一种或多种激酶，尤其抑制SYK(脾酪氨酸激酶)、LR...,"用作激酶抑制剂的吡唑并[4,3-d]嘧啶",技术领域[0001 ]本发明涉及能够抑制一种或多种激酶，尤其SYK(脾酪氨酸激酶）、LRRK...,[1.式(I)的化合物或其药学上可接受的盐其中A为NH;Ri为任选地被取代的杂烷基、C6-1...,"[US, EP, JP, KR, CN, AP, AU, BR, CA, CL, CO, C...","[{'pto': 'WO', 'patentNumber': '1998029413', '...","[{'pto': 'US', 'patentNumber': '446986', 'kind...",None,"[{'pto': 'KR', 'patentNumber': '2014002097', '...","[{'publication': 'CN103492389B', 'time': '2016...",US201161517582P
2,61fd25e5f4d57576a502166b,CN102393961A,"[王滨海, 张晶晶, 王骞, 刘俍, 王万国]",[State Grid Intelligent Technology Co Ltd],[Electric Power Research Institute of State Gr...,20110629,20120328,N/A,None,本发明公开了一种在复杂自然环境背景下，实现了基于计算机视觉的飞行器输电线路实时检测跟踪方法。...,基于计算机视觉的飞行器输电线路实时检测跟踪方法,基于计算机视觉的飞行器输电线路实时检测跟踪方法技术领域[0001]\t本发明涉及一种图像处理...,[1.一种基于计算机视觉的飞行器输电线路实时检测跟踪方法，其特征是，首先使用抗噪声的梯度图像...,[CN],None,None,"[{'pto': 'CN', 'patentNumber': '10314993', 'ki...",None,"[{'publication': 'CN102393961A', 'time': '2012...",CN201110179733.2A
3,61fd26a7f4d57576a502166d,CN103155174B,"[迈克尔·尤金·杨, 阿琼·丹尼尔·瑞尼瓦斯, 马修·R.·鲁滨逊, 亚历山大·周·米塔尔]",[Chen Hung Technology Holdings Limited],[Chen Hung Technology Holdings Ltd],20110808,20170623,N/A,"[H05K1/09, H01L31/048, G06F3/0445, H01L31/0481...",将活性或功能性添加剂嵌入到用作包括太阳能装置、智能窗、显示器等的各种电子或光电装置的组件的主...,具有表面嵌入的添加剂的装置组件和相关的制造方法,具有表面嵌入的添加剂的装置组件和相关的制造方法[0001] 相关申请的交叉引用[0002] ...,[1.一种太阳能装置，包括：前覆盖层；后覆盖层；光敏层，所述光敏层被设置在所述前覆盖层与所述...,"[US, EP, JP, KR, CN, AU, CA, WO]","[{'pto': 'CN', 'patentNumber': '10108814', 'ki...","[{'pto': 'US', 'patentNumber': '375128', 'kind...",None,"[{'pto': 'US', 'patentNumber': '9530940', 'kin...","[{'publication': 'CN103155174B', 'time': '2017...",US37168810P
4,61fd26c2f4d57576a502166f,CN103958612B,"[服部公彦, 玉井晃义, 歌崎宪, 歌崎宪一]",[Toray Industries Inc],[Toray Industries Inc],20121119,20160824,N/A,"[C08K3/04, C08L101/00, B29B7/90, B29B9/06, B29...",本发明涉及一种碳纤维增强热塑性树脂组合物及其成型品，所述成型品为使用含有(A)热塑性树脂、(...,碳纤维增强热塑性树脂组合物、该组合物的粒料和成型品,技术领域[0001] 本发明的课题在于提供一种机械特性、表面外观等优异、特别是弯曲弹性模量、...,[1.一种碳纤维增强热塑性树脂组合物，为含有(A)热塑性树脂、（B)碳纤维和(C)钛化合物的...,"[US, EP, JP, KR, CN, TW, WO]","[{'pto': 'CN', 'patentNumber': '10179613', 'ki...","[{'pto': 'JP', 'patentNumber': 'S629541', 'kin...",None,"[{'pto': 'JP', 'patentNumber': '5987335', 'kin...","[{'publication': 'CN103958612B', 'time': '2016...",JP2011-259853
5,61fd26c4f4d57576a5021671,CN102497647A,"[刘桂雄, 袁明山, 朱明武]",[South China University of Technology SCUT],[South China University of Technology SCUT],20111214,20120613,N/A,None,本发明公开了一种物联网监测系统的完整性验证评估方法，所述方法包括：采集感知层监测设备及集成终...,一种物联网监测系统的完整性验证评估方法,一种物联网监测系统的完整性验证评估方法技术领域[0001]\t本发明涉及物联网监测系统领域，...,[1.一种物联网监测系统的完整性验证评估方法，其特征在于，所述方法包括：采集感知层监测设备及...,[CN],"[{'pto': 'CN', 'patentNumber': '10112298', 'ki...",None,"[{'pto': 'CN', 'patentNumber': '10432178', 'ki...",None,"[{'publication': 'KR101702545B1', 'time': '201...",CN201110416498.6A


In [37]:
len(result)

36717

In [39]:
len(dbcn)

47313

In [40]:
len(dbus)

45096

In [43]:
# 存進MongoDB
# db.rm_patent_cn.insert_many(result.to_dict('records'))

### 切成前後五年存

前五年

CN

In [109]:
dateStr = "2010-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2015-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [105]:
db_rm_cn_5 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [108]:
# 存進MongoDB
db.rm_patent_cn_5.insert_many(db_rm_cn_5.to_dict('records'))

US

In [110]:
db_us_5 = pd.DataFrame(db_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [111]:
# 存進MongoDB
db.rm_patent_us_5.insert_many(db_us_5.to_dict('records'))

後五年

CN

In [112]:
dateStr = "2016-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2020-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [113]:
db_rm_cn_10 = pd.DataFrame(db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [114]:
# 存進MongoDB
db.rm_patent_cn_10.insert_many(db_rm_cn_10.to_dict('records'))

US

In [115]:
db_us_10 = pd.DataFrame(db_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [116]:
# 存進MongoDB
db.rm_patent_us_10.insert_many(db_us_10.to_dict('records'))

### 去除後的CPC計算

In [21]:
dateStr = "2010-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2015-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [33]:
data_assignee = db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        },
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [38]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_cn.aggregate([
        {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        },
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Jiangsu IoT Research and Development Center,H01L29,156
1,Jiangsu IoT Research and Development Center,H01L21,24
2,Jiangsu IoT Research and Development Center,66325,21
3,Jiangsu IoT Research and Development Center,G06F16,18
4,Jiangsu IoT Research and Development Center,7393,15


In [39]:
df_cpc.head()

,company,cpc,count
0,Jiangsu IoT Research and Development Center,H01L29,156
1,Jiangsu IoT Research and Development Center,H01L21,24
2,Jiangsu IoT Research and Development Center,66325,21
3,Jiangsu IoT Research and Development Center,G06F16,18
4,Jiangsu IoT Research and Development Center,7393,15


In [40]:
df_cpc.shape

(11792, 3)

In [41]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [42]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [43]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A23,A41,A42,A43,A44,A45,A46,A47,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3M innovation intellectual property Co.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3b Technology Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3d-Oxset Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A Mo Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
final_table_cn.shape

(868, 105)

In [45]:
final_table_cn.to_csv('rm_company_cpc_cn_5.csv',index=0)

In [50]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_cn_5

In [51]:
# title
conn_db_collection_rm.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

In [52]:
# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

## US

In [8]:
data_assignee = db_rm_us_5.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [9]:
len(ls_company)

705

In [10]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    try:
        data_cpc = db_rm_us_5.aggregate([
            {"$match" : {"current_assignee":{"$regex":c}}},
            {"$unwind" : "$current_assignee"},
            {"$unwind" : "$classification"},
            {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
            {"$unwind" : "$class"},
            {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
            {"$sort": {"count":-1}}
            # {"$limit":5}
        ])
    except:
        print(c)

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc_us = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc_us.head()

,company,cpc,count
0,Samsung Electronics Co Ltd,H04W4,266
1,Samsung Electronics Co Ltd,G06F3,252
2,Samsung Electronics Co Ltd,H01L27,126
3,Samsung Electronics Co Ltd,H04W52,119
4,Samsung Electronics Co Ltd,02,105


In [11]:
df_cpc_us.shape

(22211, 3)

In [12]:
# 取cpc前3碼
df_cpc_us["cpc_n"] = df_cpc_us.cpc.apply(lambda x: x[0:3])

In [13]:
# 轉成company-cpc 的總數表格
table_us = df_cpc_us.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [14]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A01,A23,A41,A42,A44,A46,A47,A61,A62,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3d-Oxides,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,1.0,0.0
2,ADVANERGY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,8.0,0.0,3.0,4.0,0.0
3,AFI Licensing LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,AFIRMA CONSULTING & TECHNOLOGIES SL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


In [15]:
final_table_us.shape

(700, 73)

In [62]:
# final_table_us.to_csv('rm_company_cpc_us_5.csv',index=0)

In [63]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_us_5

In [64]:
# title
# conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

In [65]:
# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    # conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

## 合併

In [67]:
# concat de cn
df_c2 = pd.concat([final_table_cn, final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2

cpc_n,company,A01,A23,A41,A42,A43,A44,A45,A46,A47,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3M innovation intellectual property Co.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3b Technology Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3d-Oxset Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A Mo Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,ZTE TX Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0
696,Zhejiang Shenghui Lighting Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
697,Zhongke Future Of Chip Microelectronics Techno...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
698,eBay Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0


In [68]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2

cpc_n,company,A01,A23,A41,A42,A43,A44,A45,A46,A47,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3M innovation intellectual property Co.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3b Technology Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3d-Oxides,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3d-Oxset Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1494,Zhuzhou Times Engineering Plastics Technology ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1495,Ztel Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1496,Zyray Wireless Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0
1497,eBay Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0


In [69]:
df_c2.to_csv('rm_company_cpc_cn_us_5.csv',index=0)

In [70]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_cpc_cn_us_5

In [71]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

In [72]:
# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

# 後五年

## CN-US

## 去除後的CPC計算

In [73]:
dateStr = "2016-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2020-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [74]:
data_assignee = db_rm_cn.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        },
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [75]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_cn.aggregate([
        {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        },
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Huawei Technologies Co Ltd,H04W72,1044
1,Huawei Technologies Co Ltd,H04L5,709
2,Huawei Technologies Co Ltd,H04L1,559
3,Huawei Technologies Co Ltd,00,476
4,Huawei Technologies Co Ltd,H04W76,364


In [76]:
df_cpc.head()

,company,cpc,count
0,Huawei Technologies Co Ltd,H04W72,1044
1,Huawei Technologies Co Ltd,H04L5,709
2,Huawei Technologies Co Ltd,H04L1,559
3,Huawei Technologies Co Ltd,00,476
4,Huawei Technologies Co Ltd,H04W76,364


In [77]:
df_cpc.shape

(113707, 3)

In [78]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [79]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [80]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A21,A23,A24,A41,A42,A43,A44,A45,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,1230 Intelligent Technology Haian Co ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
2,"5G IP HOLDINGS, LLC",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
3,63921 Troops of PLA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,A Field Of Shenzhen Science And Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [81]:
final_table_cn.shape

(7034, 124)

In [82]:
final_table_cn.to_csv('rm_company_cpc_cn_10.csv',index=0)

In [83]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_cn_10

In [84]:
# title
conn_db_collection_rm.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

In [85]:
# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

## US

In [32]:
data_assignee = db_rm_us_10.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [33]:
len(ls_company)

4957

In [34]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    try:
        data_cpc = db_rm_us_10.aggregate([
            {"$match" : {"current_assignee":{"$regex":c}}},
            {"$unwind" : "$current_assignee"},
            {"$unwind" : "$classification"},
            {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
            {"$unwind" : "$class"},
            {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
            {"$sort": {"count":-1}}
            # {"$limit":5}
        ])
    except:
        print(c)

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc_us = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc_us.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,8921
1,Qualcomm Inc,H04W72,7178
2,Qualcomm Inc,H04L1,3964
3,Qualcomm Inc,H04B7,3877
4,Qualcomm Inc,H04W52,2010


In [36]:
df_cpc_us.shape

(200793, 3)

In [37]:
# 取cpc前3碼
df_cpc_us["cpc_n"] = df_cpc_us.cpc.apply(lambda x: x[0:3])

In [38]:
# 轉成company-cpc 的總數表格
table_us = df_cpc_us.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [39]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
3,10757616 Canada Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,14.0,0.0,0.0,0.0,1.0,1.0,0.0
4,12a Systems Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
final_table_us.shape

(4927, 119)

In [94]:
# final_table_us.to_csv('rm_company_cpc_us_10.csv',index=0)

In [95]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_us_10

In [96]:
# title
# conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

In [97]:
# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    # conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

## 合併

In [98]:
# concat de cn
df_c2 = pd.concat([final_table_cn, final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,1230 Intelligent Technology Haian Co ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
2,"5G IP HOLDINGS, LLC",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
3,63921 Troops of PLA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,A Field Of Shenzhen Science And Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4922,incNETWORKS Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0
4923,mSignia Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
4924,nChain Holdings Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0
4925,u-blox AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,27.0,0.0,1.0,0.0,0.0


In [99]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
3,10757616 Canada Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,14.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1230 Intelligent Technology Haian Co ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11395,incNETWORKS Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0
11396,mSignia Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
11397,nChain Holdings Ltd,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,103.0,0.0,0.0,0.0,0.0
11398,u-blox AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,40.0,0.0,1.0,0.0,0.0


In [100]:
df_c2.to_csv('rm_company_cpc_cn_us_10.csv',index=0)

In [101]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_cpc_cn_us_10

In [102]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

In [103]:
# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

# DE-US

## 去除DE中priority和US重複的

In [6]:
dbde = pd.DataFrame(list(db_de.find()))

In [7]:
dbde.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,61fc144a5eb7331c4068bc4a,DE112012005419B4,"[Chi Liu, Qi Yu, Zhentan Feng, Bo Yang, Zhanwe...","[GlobalFoundries Inc, GlobalFoundries US Inc]",[GlobalFoundries Inc],20121031,20190718,N/A,"[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,SystemundVerfahrenfüreineeffizientedienstinsta...,Technisches GebietDie vorliegende Erfindung be...,[DienstinstanzenorientiertesEnergieverwaltungs...,"[US, JP, CN, CA, DE, GB, IN, WO]","[{'pto': 'CN', 'patentNumber': '10214352', 'ki...","[{'pto': 'JP', 'patentNumber': '200227860', 'k...",None,"[{'pto': 'CN', 'patentNumber': '10341505', 'ki...","[{'publication': 'DE112012005419B4', 'time': '...",CN201110456813.8A
1,61fc14c95eb7331c4068bc4c,DE102012022880A1,"[René Peter Scheurich, Junyou Pan, Amir Hossai...",[Merck Patent GmbH],[Merck Patent GmbH],20121123,20130627,N/A,"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,ElektronischeVorrichtungenenthaltendorganische...,Die vorliegende Erfindung betrifft eine elektr...,[ElektronischeVorrichtungenthaltendmindestense...,[DE],"[{'pto': 'US', 'patentNumber': '453950', 'kind...",None,"[{'pto': 'CN', 'patentNumber': '10348333', 'ki...",None,"[{'publication': 'EP3131901B1', 'time': '2018-...",EP11010151.6
2,61fc14ee5eb7331c4068bc4e,DE102012222022B4,"[Ting He, Kang-won Lee, Yonghua Lin, Qing Wang...",[International Business Machines Corp],[International Business Machines Corp],20121203,20200130,N/A,"[H04W72/1257, H04W16/14]","Verfahren, das aufweist:Empfangen von Informat...",Quasidynamischerzugriffauffrequenzenfüranwendu...,HINTERGRUNDDie vorliegende Erfindung betrifft ...,"[Verfahren,dasaufweist:EmpfangenvonInformation...","[US, JP, CN, DE, GB]","[{'pto': 'DE', 'patentNumber': '102004013494',...","[{'pto': 'US', 'patentNumber': '8326313', 'kin...",None,"[{'pto': 'US', 'patentNumber': '8909641', 'kin...","[{'publication': 'DE102012222022B4', 'time': '...","US13/328,259"
3,61fc150f5eb7331c4068bc50,DE112010006087T5,"[Kreshavan Tiruvallur, Christian Lovin, David ...",[Intel Corp],[Intel Corp],20101223,20140626,N/A,"[G06F11/2733, G06F11/36, G06F11/22, G06F11/267...",Eine Vorrichtung und ein Verfahren zum Bereits...,"ArchitekturzumTesten,zurValidierungundzurFehle...",GEBIETDiese Erfindung betrifft das Gebiet von ...,"[Vorrichtung,welcheFolgendesumfasst:einenoderm...","[US, JP, KR, CN, DE, GB, WO]",None,"[{'pto': 'JP', 'patentNumber': 'H0831559', 'ki...",None,"[{'pto': 'US', 'patentNumber': '7895565', 'kin...","[{'publication': 'DE112010006087T5', 'time': '...",PCT/US2010/061995
4,61fc15245eb7331c4068bc52,DE102012218966B4,"[Xian Wu, Keke Cai, Li Zhang, Li Jun Xia, Jian...",[International Business Machines Corp],[International Business Machines Corp],20121018,20180712,N/A,"[G06F16/951, G06F16/901]",Diese Offenbarung betrifft ein Verfahren und e...,VerfahrenundSystemzumKennzeichnenvondurchDinge...,TECHNISCHES GEBIETDie vorliegende Erfindung be...,[Verfahren(200)zumKennzeichenvondurchDingeimIn...,"[US, DE, GB]","[{'pto': 'US', 'patentNumber': '20100002082', ...","[{'pto': 'US', 'patentNumber': '6601026', 'kin...",None,"[{'pto': 'KR', 'patentNumber': '101504182', 'k...","[{'publication': 'DE102012218966B4', 'time': '...",CN2011103471559


In [8]:
dbus = pd.DataFrame(list(db_us.find()))

In [9]:
dbus.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,62009a5e7b73c16636b31082,US9990182B2,"[Prateep Misra, Arpan Pal, Balamuralidhar Puru...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20180605,20180516,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",A computing platform for intelligent developme...,Computerplatformfordevelopmentanddeploymentofs...,The present application claims the benefit of ...,[1.Anintelligenttransportationsystemcomprising...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'US', 'patentNumber': '527662', 'kind...","[{'pto': 'JP', 'patentNumber': '200126069', 'k...","[{'pto': 'US', 'patentNumber': '10965787', 'ki...","[{'pto': 'US', 'patentNumber': '9932033', 'kin...","[{'publication': 'US9990182B2', 'time': '2018-...",IN2651/MUM/2011
1,62009a807b73c16636b31084,US20190027414A1,"[Senthil Kumar Ramadas, Saravanan Shanmugavel]","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...",20180924,20190124,N/A,"[B32B7/02, H01L23/29, B82Y30/00, C23C28/00, C2...","Disclosed is an encapsulation barrier stack, c...",Encapsulationbarrierstack,The present application is a divisional applic...,[1.Amethodofmanufacturinganencapsulationbarrie...,"[US, EP, JP, KR, CN, AU, CA, DK, MX, MY, RU, S...","[{'pto': 'US', 'patentNumber': '20100089636', ...","[{'pto': 'US', 'patentNumber': '560778', 'kind...","[{'pto': 'US', 'patentNumber': '10854853', 'ki...","[{'pto': 'US', 'patentNumber': '20150314941', ...","[{'publication': 'US20190027414A1', 'time': '2...",US201161550764P
2,62009aea7b73c16636b31086,US10216485B2,"[Prateep Misra, Arpan Pal, Balamuralidhar Puru...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20190226,20190206,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",A method and system for real-time analytics of...,Computerplatformfordevelopmentanddeploymentofs...,The present application claims the benefit of ...,[1.Anintelligenttransportationsystemcomprising...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'US', 'patentNumber': '20020128751', ...","[{'pto': 'US', 'patentNumber': '7779383', 'kin...","[{'pto': 'US', 'patentNumber': '20180373418', ...","[{'pto': 'US', 'patentNumber': '8250213', 'kin...","[{'publication': 'US10216485B2', 'time': '2019...",IN2651/MUM/2011
3,62009b557b73c16636b31088,US20140310243A1,"[Steven James McGee, Tammy Lee McGee]",[],[Mr. Steven James McGee],20120813,20141016,N/A,"[G06F17/30575, G06F16/27, G06Q10/101, G06Q10/1...","Systemic, adaptive procedural template compris...",Heartbeaconcycle,Related applications by Applicant Steven J. Mc...,"[1.Asystemic,adaptiveproceduraltemplatemethodu...",[US],"[{'pto': 'US', 'patentNumber': '20060217138', ...",None,"[{'pto': 'US', 'patentNumber': '20130086129', ...",None,"[{'publication': 'US20140310243A1', 'time': '2...","US12/856,715"
4,62009b717b73c16636b3108a,US8965375B2,"[Juergen Michel, Dirk Rose]",[Nokia Solutions and Networks Oy],[Nokia Solutions and Networks Oy],20110504,20150224,20150204,"[H04W36/04, H04W24/10, H04W52/0206, H04W52/023...",There are provided measures for a pathloss-bas...,Pathlossbasedaccessnodewake-upcontrol,The present invention relates to a pathloss-ba...,[1.Amethodcomprisingretrievinganaveragehandove...,"[US, EP, WO]","[{'pto': 'US', 'patentNumber': '6498932', 'kin...",None,"[{'pto': 'US', 'patentNumber': '20150043538', ...","[{'pto': 'US', 'patentNumber': '9203489', 'kin...","[{'publication': 'US8965375B2', 'time': '2015-...",PCT/EP2011/057133


In [10]:
result = pd.concat([dbde,dbus,dbus]).drop_duplicates(subset=['priority'],keep=False)

In [11]:
result.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,61fc144a5eb7331c4068bc4a,DE112012005419B4,"[Chi Liu, Qi Yu, Zhentan Feng, Bo Yang, Zhanwe...","[GlobalFoundries Inc, GlobalFoundries US Inc]",[GlobalFoundries Inc],20121031,20190718,N/A,"[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,SystemundVerfahrenfüreineeffizientedienstinsta...,Technisches GebietDie vorliegende Erfindung be...,[DienstinstanzenorientiertesEnergieverwaltungs...,"[US, JP, CN, CA, DE, GB, IN, WO]","[{'pto': 'CN', 'patentNumber': '10214352', 'ki...","[{'pto': 'JP', 'patentNumber': '200227860', 'k...",None,"[{'pto': 'CN', 'patentNumber': '10341505', 'ki...","[{'publication': 'DE112012005419B4', 'time': '...",CN201110456813.8A
1,61fc14c95eb7331c4068bc4c,DE102012022880A1,"[René Peter Scheurich, Junyou Pan, Amir Hossai...",[Merck Patent GmbH],[Merck Patent GmbH],20121123,20130627,N/A,"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,ElektronischeVorrichtungenenthaltendorganische...,Die vorliegende Erfindung betrifft eine elektr...,[ElektronischeVorrichtungenthaltendmindestense...,[DE],"[{'pto': 'US', 'patentNumber': '453950', 'kind...",None,"[{'pto': 'CN', 'patentNumber': '10348333', 'ki...",None,"[{'publication': 'EP3131901B1', 'time': '2018-...",EP11010151.6
4,61fc15245eb7331c4068bc52,DE102012218966B4,"[Xian Wu, Keke Cai, Li Zhang, Li Jun Xia, Jian...",[International Business Machines Corp],[International Business Machines Corp],20121018,20180712,N/A,"[G06F16/951, G06F16/901]",Diese Offenbarung betrifft ein Verfahren und e...,VerfahrenundSystemzumKennzeichnenvondurchDinge...,TECHNISCHES GEBIETDie vorliegende Erfindung be...,[Verfahren(200)zumKennzeichenvondurchDingeimIn...,"[US, DE, GB]","[{'pto': 'US', 'patentNumber': '20100002082', ...","[{'pto': 'US', 'patentNumber': '6601026', 'kin...",None,"[{'pto': 'KR', 'patentNumber': '101504182', 'k...","[{'publication': 'DE102012218966B4', 'time': '...",CN2011103471559
6,61fc15495eb7331c4068bc56,DE112012005533B4,"[Li Li, Qi Yu, Chi Liu, Juwei Shi]",[International Business Machines Corp],[International Business Machines Corp],20121031,20200618,N/A,"[G06F16/245, G06F16/2255, G06F16/951]",Die vorliegende Erfindung möchte ein Verfahren...,UnterstützendeAbfrageundeinAbfragen,GEBIET DER ERFINDUNGAusführungsformen der vorl...,"[Verfahren,aufweisend:KonstruiereneinesSpeiche...","[US, JP, CN, DE, GB, WO]",None,"[{'pto': 'US', 'patentNumber': '599171', 'kind...",None,"[{'pto': 'US', 'patentNumber': '8151327', 'kin...","[{'publication': 'DE112012005533B4', 'time': '...",CN201110461370.1
19,61fc16ae5eb7331c4068bc70,DE102013015167A1,"[Wen Xu, Xiaojun Ma]",[Intel IP Corp],[Intel IP Corp],20130911,20150312,N/A,"[H04W24/08, H04L43/028, H04L47/27, H04W48/16, ...",Ein Verfahren (900) beinhaltet das Empfangen (...,VerfahrenzumFilternvonKandidatenzellen,GEBIET DER ERFINDUNGDie Patentanmeldung bezieh...,"[Verfahren(900),welchesumfasst:Empfangen(901)e...","[US, DE]","[{'pto': 'US', 'patentNumber': '8027329', 'kin...","[{'pto': 'JP', 'patentNumber': '4786503', 'kin...",None,"[{'pto': 'US', 'patentNumber': '20160105883', ...","[{'publication': 'DE102013015167B4', 'time': '...",DE102013015167.8A


In [12]:
len(result)

1148

In [14]:
len(dbde)

1833

In [15]:
len(dbus)

45096

In [16]:
# 存進MongoDB
db.rm_patent_de.insert_many(result.to_dict('records'))

## 切成前後五年存

前五年

DE

In [18]:
dateStr = "2010-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2015-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [19]:
db_rm_de_5 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [20]:
# 存進MongoDB
# db.rm_patent_de_5.insert_many(db_rm_de_5.to_dict('records'))

後五年

DE

In [21]:
dateStr = "2016-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2020-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [22]:
db_rm_de_10 = pd.DataFrame(db_rm_de.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [23]:
# 存進MongoDB
# db.rm_patent_de_10.insert_many(db_rm_de_10.to_dict('records'))

## 前五年

In [21]:
# dateStr = "2010-01-01T00:00:00.000Z"
# start = dateutil.parser.parse(dateStr)

# dateEnd = "2015-12-31T00:00:00.000Z"
# end = dateutil.parser.parse(dateEnd)

In [16]:
data_assignee = db_rm_de_5.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [17]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_de_5.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Robert Bosch GmbH,12,7
1,Robert Bosch GmbH,H04L9,7
2,Robert Bosch GmbH,F01N2610,7
3,Robert Bosch GmbH,G08G1,7
4,Robert Bosch GmbH,00,5


In [18]:
df_cpc.head()

,company,cpc,count
0,Robert Bosch GmbH,12,7
1,Robert Bosch GmbH,H04L9,7
2,Robert Bosch GmbH,F01N2610,7
3,Robert Bosch GmbH,G08G1,7
4,Robert Bosch GmbH,00,5


In [19]:
df_cpc.shape

(646, 3)

In [20]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [21]:
# 轉成company-cpc 的總數表格
table_de = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [24]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,A42,A61,B05,B27,B41,B42,B81,C07,C08,...,G07,G08,G09,H01,H02,H03,H04,H05,Y02,Y10
0,Advantech Service IoT GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,24.0,0.0,0.0,3.0,0.0,0.0,0.0
1,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
2,Armstrong DLW GmbH,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Audi AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DLOG GES fur ELEKTRONISCHE DATENTECHNIK MBH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,24.0,0.0,0.0,3.0,0.0,0.0,0.0


In [25]:
final_table_de.shape

(41, 29)

In [33]:
# final_table_de.to_csv('rm_company_cpc_de_5.csv',index=0)

In [34]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_de_5

In [35]:
# title
# conn_db_collection_rm.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

In [36]:
# 內容
for k in final_table_cn.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    # conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### 合併

In [26]:
# concat de cn
df_c2 = pd.concat([final_table_de, final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2

cpc_n,company,A01,A23,A41,A42,A44,A46,A47,A61,A62,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,Advantech Service IoT GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,24.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
1,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
2,Armstrong DLW GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Audi AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DLOG GES fur ELEKTRONISCHE DATENTECHNIK MBH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,24.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,ZTE TX Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0
696,Zhejiang Shenghui Lighting Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
697,Zhongke Future Of Chip Microelectronics Techno...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
698,eBay Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0


In [27]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2

cpc_n,company,A01,A23,A41,A42,A44,A46,A47,A61,A62,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3d-Oxides,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,1.0,0.0
2,ADVANERGY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,8.0,0.0,3.0,4.0,0.0
3,AFI Licensing LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,AFIRMA CONSULTING & TECHNOLOGIES SL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722,ZTE TX Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0
723,Zhejiang Shenghui Lighting Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
724,Zhongke Future Of Chip Microelectronics Techno...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
725,eBay Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0


In [28]:
df_c2.to_csv('rm_company_cpc_de_us_5.csv',index=0)

In [29]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_cpc_de_us_5

In [30]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

In [31]:
# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

## 後五年

In [73]:
# dateStr = "2016-01-01T00:00:00.000Z"
# start = dateutil.parser.parse(dateStr)

# dateEnd = "2020-12-31T00:00:00.000Z"
# end = dateutil.parser.parse(dateEnd)

In [41]:
data_assignee = db_rm_de_10.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [42]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_de_10.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Intel Corp,G06F9,186
1,Intel Corp,G06N3,82
2,Intel Corp,G06F12,53
3,Intel Corp,00,52
4,Intel Corp,G06F3,51


In [43]:
df_cpc.head()

,company,cpc,count
0,Intel Corp,G06F9,186
1,Intel Corp,G06N3,82
2,Intel Corp,G06F12,53
3,Intel Corp,00,52
4,Intel Corp,G06F3,51


In [44]:
df_cpc.shape

(8553, 3)

In [45]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [46]:
# 轉成company-cpc 的總數表格
table_de = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [47]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,A01,A24,A41,A42,A44,A45,A47,A61,A62,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,ALBA GROUP PLC & Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,AT&S Austria Technologie and Systemtechnik AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0
3,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Accell Group NV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
final_table_de.shape

(328, 74)

In [45]:
# final_table_de.to_csv('rm_company_cpc_de_10.csv',index=0)

In [46]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_de_10

In [47]:
# title
# conn_db_collection_rm.insert_one({"company":final_table_cn.columns.tolist()[0],"cpc":final_table_cn.columns.tolist()[1:]})

In [48]:
# 內容
for k in final_table_de.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    # conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### 合併

In [49]:
# concat de cn
df_c2 = pd.concat([final_table_de, final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,ALBA GROUP PLC & Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,AT&S Austria Technologie and Systemtechnik AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0
3,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Accell Group NV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4922,incNETWORKS Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0
4923,mSignia Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
4924,nChain Holdings Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0
4925,u-blox AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,27.0,0.0,1.0,0.0,0.0


In [50]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
3,10757616 Canada Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,14.0,0.0,0.0,0.0,1.0,1.0,0.0
4,12a Systems Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5123,incNETWORKS Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0
5124,mSignia Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
5125,nChain Holdings Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0
5126,u-blox AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,27.0,0.0,1.0,0.0,0.0


In [51]:
df_c2.to_csv('rm_company_cpc_de_us_10.csv',index=0)

In [52]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_cpc_de_us_10

In [53]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

In [54]:
# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

# US-CN-DE

## 去除US中priority和DE重複的

In [6]:
dbde = pd.DataFrame(list(db_rm_de.find()))

In [7]:
dbde.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,61fc144a5eb7331c4068bc4a,DE112012005419B4,"[Chi Liu, Qi Yu, Zhentan Feng, Bo Yang, Zhanwe...","[GlobalFoundries Inc, GlobalFoundries US Inc]",[GlobalFoundries Inc],20121031,20190718,N/A,"[G06F1/3234, G06F1/329, H04W52/0216, H04L43/08...",None,SystemundVerfahrenfüreineeffizientedienstinsta...,Technisches GebietDie vorliegende Erfindung be...,[DienstinstanzenorientiertesEnergieverwaltungs...,"[US, JP, CN, CA, DE, GB, IN, WO]","[{'pto': 'CN', 'patentNumber': '10214352', 'ki...","[{'pto': 'JP', 'patentNumber': '200227860', 'k...",None,"[{'pto': 'CN', 'patentNumber': '10341505', 'ki...","[{'publication': 'DE112012005419B4', 'time': '...",CN201110456813.8A
1,61fc14c95eb7331c4068bc4c,DE102012022880A1,"[René Peter Scheurich, Junyou Pan, Amir Hossai...",[Merck Patent GmbH],[Merck Patent GmbH],20121123,20130627,N/A,"[C07D417/04, C07D403/04, C07D403/12, C07D403/1...",Die vorliegende Erfindung betrifft eine elektr...,ElektronischeVorrichtungenenthaltendorganische...,Die vorliegende Erfindung betrifft eine elektr...,[ElektronischeVorrichtungenthaltendmindestense...,[DE],"[{'pto': 'US', 'patentNumber': '453950', 'kind...",None,"[{'pto': 'CN', 'patentNumber': '10348333', 'ki...",None,"[{'publication': 'EP3131901B1', 'time': '2018-...",EP11010151.6
2,61fc15245eb7331c4068bc52,DE102012218966B4,"[Xian Wu, Keke Cai, Li Zhang, Li Jun Xia, Jian...",[International Business Machines Corp],[International Business Machines Corp],20121018,20180712,N/A,"[G06F16/951, G06F16/901]",Diese Offenbarung betrifft ein Verfahren und e...,VerfahrenundSystemzumKennzeichnenvondurchDinge...,TECHNISCHES GEBIETDie vorliegende Erfindung be...,[Verfahren(200)zumKennzeichenvondurchDingeimIn...,"[US, DE, GB]","[{'pto': 'US', 'patentNumber': '20100002082', ...","[{'pto': 'US', 'patentNumber': '6601026', 'kin...",None,"[{'pto': 'KR', 'patentNumber': '101504182', 'k...","[{'publication': 'DE102012218966B4', 'time': '...",CN2011103471559
3,61fc15495eb7331c4068bc56,DE112012005533B4,"[Li Li, Qi Yu, Chi Liu, Juwei Shi]",[International Business Machines Corp],[International Business Machines Corp],20121031,20200618,N/A,"[G06F16/245, G06F16/2255, G06F16/951]",Die vorliegende Erfindung möchte ein Verfahren...,UnterstützendeAbfrageundeinAbfragen,GEBIET DER ERFINDUNGAusführungsformen der vorl...,"[Verfahren,aufweisend:KonstruiereneinesSpeiche...","[US, JP, CN, DE, GB, WO]",None,"[{'pto': 'US', 'patentNumber': '599171', 'kind...",None,"[{'pto': 'US', 'patentNumber': '8151327', 'kin...","[{'publication': 'DE112012005533B4', 'time': '...",CN201110461370.1
4,61fc16ae5eb7331c4068bc70,DE102013015167A1,"[Wen Xu, Xiaojun Ma]",[Intel IP Corp],[Intel IP Corp],20130911,20150312,N/A,"[H04W24/08, H04L43/028, H04L47/27, H04W48/16, ...",Ein Verfahren (900) beinhaltet das Empfangen (...,VerfahrenzumFilternvonKandidatenzellen,GEBIET DER ERFINDUNGDie Patentanmeldung bezieh...,"[Verfahren(900),welchesumfasst:Empfangen(901)e...","[US, DE]","[{'pto': 'US', 'patentNumber': '8027329', 'kin...","[{'pto': 'JP', 'patentNumber': '4786503', 'kin...",None,"[{'pto': 'US', 'patentNumber': '20160105883', ...","[{'publication': 'DE102013015167B4', 'time': '...",DE102013015167.8A


In [8]:
dbus = pd.DataFrame(list(db_us.find()))

In [9]:
dbus.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,62009a5e7b73c16636b31082,US9990182B2,"[Prateep Misra, Arpan Pal, Balamuralidhar Puru...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20180605,20180516,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",A computing platform for intelligent developme...,Computerplatformfordevelopmentanddeploymentofs...,The present application claims the benefit of ...,[1.Anintelligenttransportationsystemcomprising...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'US', 'patentNumber': '527662', 'kind...","[{'pto': 'JP', 'patentNumber': '200126069', 'k...","[{'pto': 'US', 'patentNumber': '10965787', 'ki...","[{'pto': 'US', 'patentNumber': '9932033', 'kin...","[{'publication': 'US9990182B2', 'time': '2018-...",IN2651/MUM/2011
1,62009a807b73c16636b31084,US20190027414A1,"[Senthil Kumar Ramadas, Saravanan Shanmugavel]","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...",20180924,20190124,N/A,"[B32B7/02, H01L23/29, B82Y30/00, C23C28/00, C2...","Disclosed is an encapsulation barrier stack, c...",Encapsulationbarrierstack,The present application is a divisional applic...,[1.Amethodofmanufacturinganencapsulationbarrie...,"[US, EP, JP, KR, CN, AU, CA, DK, MX, MY, RU, S...","[{'pto': 'US', 'patentNumber': '20100089636', ...","[{'pto': 'US', 'patentNumber': '560778', 'kind...","[{'pto': 'US', 'patentNumber': '10854853', 'ki...","[{'pto': 'US', 'patentNumber': '20150314941', ...","[{'publication': 'US20190027414A1', 'time': '2...",US201161550764P
2,62009aea7b73c16636b31086,US10216485B2,"[Prateep Misra, Arpan Pal, Balamuralidhar Puru...",[Tata Consultancy Services Ltd],[Tata Consultancy Services Ltd],20120918,20190226,20190206,"[G06F8/00, G06F11/3668, G06F8/36, G06F8/60, H0...",A method and system for real-time analytics of...,Computerplatformfordevelopmentanddeploymentofs...,The present application claims the benefit of ...,[1.Anintelligenttransportationsystemcomprising...,"[US, EP, JP, CN, AU, CA, MX, TW, WO, ZA]","[{'pto': 'US', 'patentNumber': '20020128751', ...","[{'pto': 'US', 'patentNumber': '7779383', 'kin...","[{'pto': 'US', 'patentNumber': '20180373418', ...","[{'pto': 'US', 'patentNumber': '8250213', 'kin...","[{'publication': 'US10216485B2', 'time': '2019...",IN2651/MUM/2011
3,62009b557b73c16636b31088,US20140310243A1,"[Steven James McGee, Tammy Lee McGee]",[],[Mr. Steven James McGee],20120813,20141016,N/A,"[G06F17/30575, G06F16/27, G06Q10/101, G06Q10/1...","Systemic, adaptive procedural template compris...",Heartbeaconcycle,Related applications by Applicant Steven J. Mc...,"[1.Asystemic,adaptiveproceduraltemplatemethodu...",[US],"[{'pto': 'US', 'patentNumber': '20060217138', ...",None,"[{'pto': 'US', 'patentNumber': '20130086129', ...",None,"[{'publication': 'US20140310243A1', 'time': '2...","US12/856,715"
4,62009b717b73c16636b3108a,US8965375B2,"[Juergen Michel, Dirk Rose]",[Nokia Solutions and Networks Oy],[Nokia Solutions and Networks Oy],20110504,20150224,20150204,"[H04W36/04, H04W24/10, H04W52/0206, H04W52/023...",There are provided measures for a pathloss-bas...,Pathlossbasedaccessnodewake-upcontrol,The present invention relates to a pathloss-ba...,[1.Amethodcomprisingretrievinganaveragehandove...,"[US, EP, WO]","[{'pto': 'US', 'patentNumber': '6498932', 'kin...",None,"[{'pto': 'US', 'patentNumber': '20150043538', ...","[{'pto': 'US', 'patentNumber': '9203489', 'kin...","[{'publication': 'US8965375B2', 'time': '2015-...",PCT/EP2011/057133


In [10]:
result = pd.concat([dbus,dbde,dbde]).drop_duplicates(subset=['priority'],keep=False)

In [11]:
result.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
1,62009a807b73c16636b31084,US20190027414A1,"[Senthil Kumar Ramadas, Saravanan Shanmugavel]","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...",20180924,20190124,N/A,"[B32B7/02, H01L23/29, B82Y30/00, C23C28/00, C2...","Disclosed is an encapsulation barrier stack, c...",Encapsulationbarrierstack,The present application is a divisional applic...,[1.Amethodofmanufacturinganencapsulationbarrie...,"[US, EP, JP, KR, CN, AU, CA, DK, MX, MY, RU, S...","[{'pto': 'US', 'patentNumber': '20100089636', ...","[{'pto': 'US', 'patentNumber': '560778', 'kind...","[{'pto': 'US', 'patentNumber': '10854853', 'ki...","[{'pto': 'US', 'patentNumber': '20150314941', ...","[{'publication': 'US20190027414A1', 'time': '2...",US201161550764P
3,62009b557b73c16636b31088,US20140310243A1,"[Steven James McGee, Tammy Lee McGee]",[],[Mr. Steven James McGee],20120813,20141016,N/A,"[G06F17/30575, G06F16/27, G06Q10/101, G06Q10/1...","Systemic, adaptive procedural template compris...",Heartbeaconcycle,Related applications by Applicant Steven J. Mc...,"[1.Asystemic,adaptiveproceduraltemplatemethodu...",[US],"[{'pto': 'US', 'patentNumber': '20060217138', ...",None,"[{'pto': 'US', 'patentNumber': '20130086129', ...",None,"[{'publication': 'US20140310243A1', 'time': '2...","US12/856,715"
4,62009b717b73c16636b3108a,US8965375B2,"[Juergen Michel, Dirk Rose]",[Nokia Solutions and Networks Oy],[Nokia Solutions and Networks Oy],20110504,20150224,20150204,"[H04W36/04, H04W24/10, H04W52/0206, H04W52/023...",There are provided measures for a pathloss-bas...,Pathlossbasedaccessnodewake-upcontrol,The present invention relates to a pathloss-ba...,[1.Amethodcomprisingretrievinganaveragehandove...,"[US, EP, WO]","[{'pto': 'US', 'patentNumber': '6498932', 'kin...",None,"[{'pto': 'US', 'patentNumber': '20150043538', ...","[{'pto': 'US', 'patentNumber': '9203489', 'kin...","[{'publication': 'US8965375B2', 'time': '2015-...",PCT/EP2011/057133
5,62009b8d7b73c16636b3108c,US8929880B2,"[Rapeepat Ratasuk, Amitabha Ghosh, Weidong Yang]",[Motorola Solutions Inc],[Motorola Solutions Inc],20110421,20150106,20141217,"[H04W52/244, H04L1/0025, H04W52/146, H04W52/283]",None,Uplinkinterferencemanagementforaheterogeneousw...,The invention generally relates to uplink inte...,[1.Amethodformanaginginterferencefromuserequip...,"[US, EP, WO]","[{'pto': 'US', 'patentNumber': '559494', 'kind...",None,"[{'pto': 'US', 'patentNumber': '10757655', 'ki...","[{'pto': 'US', 'patentNumber': '20110090820', ...","[{'publication': 'US8929880B2', 'time': '2015-...","US13/092,057"
6,62009bab7b73c16636b3108e,US10602420B2,"[Chan-Ho Min, Jung-Soo Jung, Dong-Wook Kim, Ky...",[Samsung Electronics Co Ltd],[Samsung Electronics Co Ltd],20160127,20200324,20200304,"[H04W36/32, H04W36/0055, H04W36/0058, H04W36/0...",An apparatus and method for providing handover...,Apparatusandmethodforprovidinghandoversupporti...,This application is a continuation application...,[1.Amethodforoperatingaservingbasestation(BS)i...,"[US, EP, JP, KR, CN, WO]","[{'pto': 'KR', 'patentNumber': '2004008154', '...","[{'pto': 'WO', 'patentNumber': '2008151549', '...",None,"[{'pto': 'JP', 'patentNumber': '5174580', 'kin...","[{'publication': 'US10966129B2', 'time': '2021...",KR1020100038886A


In [12]:
len(result)

41951

In [13]:
len(dbde)

1148

In [14]:
len(dbus)

45096

In [15]:
dbcn = pd.DataFrame(list(db_rm_cn.find()))
dbcn

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
0,61fd25e1f4d57576a5021669,CN103492389B,"[M·阿尔姆施泰特尔, M·托尔曼, A·特雷姆尔, N·特劳贝]",[Origenis GmbH],[Origenis GmbH],20120423,20160914,N/A,"[C07D487/04, A61K31/519, A61P1/00, A61P11/00, ...",本发明涉及式(I)的新化合物，其能够抑制一种或多种激酶，尤其抑制SYK(脾酪氨酸激酶)、LR...,"用作激酶抑制剂的吡唑并[4,3-d]嘧啶",技术领域[0001 ]本发明涉及能够抑制一种或多种激酶，尤其SYK(脾酪氨酸激酶）、LRRK...,[1.式(I)的化合物或其药学上可接受的盐其中A为NH;Ri为任选地被取代的杂烷基、C6-1...,"[US, EP, JP, KR, CN, AP, AU, BR, CA, CL, CO, C...","[{'pto': 'WO', 'patentNumber': '1998029413', '...","[{'pto': 'US', 'patentNumber': '446986', 'kind...",None,"[{'pto': 'KR', 'patentNumber': '2014002097', '...","[{'publication': 'CN103492389B', 'time': '2016...",US201161517582P
1,61fd25e5f4d57576a502166b,CN102393961A,"[王滨海, 张晶晶, 王骞, 刘俍, 王万国]",[State Grid Intelligent Technology Co Ltd],[Electric Power Research Institute of State Gr...,20110629,20120328,N/A,None,本发明公开了一种在复杂自然环境背景下，实现了基于计算机视觉的飞行器输电线路实时检测跟踪方法。...,基于计算机视觉的飞行器输电线路实时检测跟踪方法,基于计算机视觉的飞行器输电线路实时检测跟踪方法技术领域[0001]\t本发明涉及一种图像处理...,[1.一种基于计算机视觉的飞行器输电线路实时检测跟踪方法，其特征是，首先使用抗噪声的梯度图像...,[CN],None,None,"[{'pto': 'CN', 'patentNumber': '10314993', 'ki...",None,"[{'publication': 'CN102393961A', 'time': '2012...",CN201110179733.2A
2,61fd26a7f4d57576a502166d,CN103155174B,"[迈克尔·尤金·杨, 阿琼·丹尼尔·瑞尼瓦斯, 马修·R.·鲁滨逊, 亚历山大·周·米塔尔]",[Chen Hung Technology Holdings Limited],[Chen Hung Technology Holdings Ltd],20110808,20170623,N/A,"[H05K1/09, H01L31/048, G06F3/0445, H01L31/0481...",将活性或功能性添加剂嵌入到用作包括太阳能装置、智能窗、显示器等的各种电子或光电装置的组件的主...,具有表面嵌入的添加剂的装置组件和相关的制造方法,具有表面嵌入的添加剂的装置组件和相关的制造方法[0001] 相关申请的交叉引用[0002] ...,[1.一种太阳能装置，包括：前覆盖层；后覆盖层；光敏层，所述光敏层被设置在所述前覆盖层与所述...,"[US, EP, JP, KR, CN, AU, CA, WO]","[{'pto': 'CN', 'patentNumber': '10108814', 'ki...","[{'pto': 'US', 'patentNumber': '375128', 'kind...",None,"[{'pto': 'US', 'patentNumber': '9530940', 'kin...","[{'publication': 'CN103155174B', 'time': '2017...",US37168810P
3,61fd26c2f4d57576a502166f,CN103958612B,"[服部公彦, 玉井晃义, 歌崎宪, 歌崎宪一]",[Toray Industries Inc],[Toray Industries Inc],20121119,20160824,N/A,"[C08K3/04, C08L101/00, B29B7/90, B29B9/06, B29...",本发明涉及一种碳纤维增强热塑性树脂组合物及其成型品，所述成型品为使用含有(A)热塑性树脂、(...,碳纤维增强热塑性树脂组合物、该组合物的粒料和成型品,技术领域[0001] 本发明的课题在于提供一种机械特性、表面外观等优异、特别是弯曲弹性模量、...,[1.一种碳纤维增强热塑性树脂组合物，为含有(A)热塑性树脂、（B)碳纤维和(C)钛化合物的...,"[US, EP, JP, KR, CN, TW, WO]","[{'pto': 'CN', 'patentNumber': '10179613', 'ki...","[{'pto': 'JP', 'patentNumber': 'S629541', 'kin...",None,"[{'pto': 'JP', 'patentNumber': '5987335', 'kin...","[{'publication': 'CN103958612B', 'time': '2016...",JP2011-259853
4,61fd26c4f4d57576a5021671,CN102497647A,"[刘桂雄, 袁明山, 朱明武]",[South China University of Technology SCUT],[South China University of Technology SCUT],20111214,20120613,N/A,None,本发明公开了一种物联网监测系统的完整性验证评估方法，所述方法包括：采集感知层监测设备及集成终...,一种物联网监测系统的完整性验证评估方法,一种物联网监测系统的完整性验证评估方法技术领域[0001]\t本发明涉及物联网监测系统领域，...,[1.一种物联网监测系统的完整性验证评估方法，其特征在于，所述方法包括：采集感知层监测设备及...,[CN],"[{'pto': 'CN', 'patentNumber': '10112298', 'ki...",None,"[{'pto': 'CN', 'patentNumber': '10432178', 'ki...",None,"[{'publication': 'KR101702545B1', 'time': '201...",CN201110416498.6A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36712,620072f1f4d57576a50387ff,CN111161757B,"[莫凡, 孙珏, 刘士杰]",[Mgjia Beijing Technology Co ltd],[Mgjia Beijing Technology Co ltd],20191227,20210903,N/A,"[G10L25/48, G01S5/24, G10L21/0272, G10L21/028,...",本公开涉及一种声源定位方法、装置、可读存储介质及电子设备。该方法包括：从N个麦克风中获取目标...,声源定位方法、装置、可读存储介质及电子设备,技术领域本公开涉及定位技术领域，具体地，涉及一种声源定位方法、装置、可读存储介质及电子设备。...,[1.一种声源定位方法，其特征在于，包括：从N个麦克风中获取目标音频信号，其中，每一所述麦克...,[CN],"[{'pto': 'EP', 'patentNumber': '2530484', 'kin...","[{'pto': 'US', 'patentNumber': '9788109', 'kin...",None,None,"[{'publication': 'KR20190024762A', 'time': '20...",CN201911373874.0A
36713,620072f3f4d57576a5038801,CN111218903B,"[严兆彬, 邓怒涛, 刘志强, 余岸阳, 王飞]",[Aisino Corp],[Aisino Corp],20191231,20210928,N/A,[E01F13/06],本公开涉及一种闸机通道拍打门

In [16]:
result2 = pd.concat([result,dbcn,dbcn]).drop_duplicates(subset=['priority'],keep=False)

In [17]:
result2.head()

,_id,patentNumber,inventorName,current_assignee,original_assignee,appDate,pubDate,graDate,classification,brief,title,description,claim,countrystatus,patentCitation,family_to_family_citing,cited,family_to_family_cited,similarDocuments,priority
1,62009a807b73c16636b31084,US20190027414A1,"[Senthil Kumar Ramadas, Saravanan Shanmugavel]","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...","[TERA-BARRIER FILMS Pte Ltd, Agency for Scienc...",20180924,20190124,N/A,"[B32B7/02, H01L23/29, B82Y30/00, C23C28/00, C2...","Disclosed is an encapsulation barrier stack, c...",Encapsulationbarrierstack,The present application is a divisional applic...,[1.Amethodofmanufacturinganencapsulationbarrie...,"[US, EP, JP, KR, CN, AU, CA, DK, MX, MY, RU, S...","[{'pto': 'US', 'patentNumber': '20100089636', ...","[{'pto': 'US', 'patentNumber': '560778', 'kind...","[{'pto': 'US', 'patentNumber': '10854853', 'ki...","[{'pto': 'US', 'patentNumber': '20150314941', ...","[{'publication': 'US20190027414A1', 'time': '2...",US201161550764P
3,62009b557b73c16636b31088,US20140310243A1,"[Steven James McGee, Tammy Lee McGee]",[],[Mr. Steven James McGee],20120813,20141016,N/A,"[G06F17/30575, G06F16/27, G06Q10/101, G06Q10/1...","Systemic, adaptive procedural template compris...",Heartbeaconcycle,Related applications by Applicant Steven J. Mc...,"[1.Asystemic,adaptiveproceduraltemplatemethodu...",[US],"[{'pto': 'US', 'patentNumber': '20060217138', ...",None,"[{'pto': 'US', 'patentNumber': '20130086129', ...",None,"[{'publication': 'US20140310243A1', 'time': '2...","US12/856,715"
4,62009b717b73c16636b3108a,US8965375B2,"[Juergen Michel, Dirk Rose]",[Nokia Solutions and Networks Oy],[Nokia Solutions and Networks Oy],20110504,20150224,20150204,"[H04W36/04, H04W24/10, H04W52/0206, H04W52/023...",There are provided measures for a pathloss-bas...,Pathlossbasedaccessnodewake-upcontrol,The present invention relates to a pathloss-ba...,[1.Amethodcomprisingretrievinganaveragehandove...,"[US, EP, WO]","[{'pto': 'US', 'patentNumber': '6498932', 'kin...",None,"[{'pto': 'US', 'patentNumber': '20150043538', ...","[{'pto': 'US', 'patentNumber': '9203489', 'kin...","[{'publication': 'US8965375B2', 'time': '2015-...",PCT/EP2011/057133
5,62009b8d7b73c16636b3108c,US8929880B2,"[Rapeepat Ratasuk, Amitabha Ghosh, Weidong Yang]",[Motorola Solutions Inc],[Motorola Solutions Inc],20110421,20150106,20141217,"[H04W52/244, H04L1/0025, H04W52/146, H04W52/283]",None,Uplinkinterferencemanagementforaheterogeneousw...,The invention generally relates to uplink inte...,[1.Amethodformanaginginterferencefromuserequip...,"[US, EP, WO]","[{'pto': 'US', 'patentNumber': '559494', 'kind...",None,"[{'pto': 'US', 'patentNumber': '10757655', 'ki...","[{'pto': 'US', 'patentNumber': '20110090820', ...","[{'publication': 'US8929880B2', 'time': '2015-...","US13/092,057"
6,62009bab7b73c16636b3108e,US10602420B2,"[Chan-Ho Min, Jung-Soo Jung, Dong-Wook Kim, Ky...",[Samsung Electronics Co Ltd],[Samsung Electronics Co Ltd],20160127,20200324,20200304,"[H04W36/32, H04W36/0055, H04W36/0058, H04W36/0...",An apparatus and method for providing handover...,Apparatusandmethodforprovidinghandoversupporti...,This application is a continuation application...,[1.Amethodforoperatingaservingbasestation(BS)i...,"[US, EP, JP, KR, CN, WO]","[{'pto': 'KR', 'patentNumber': '2004008154', '...","[{'pto': 'WO', 'patentNumber': '2008151549', '...",None,"[{'pto': 'JP', 'patentNumber': '5174580', 'kin...","[{'publication': 'US10966129B2', 'time': '2021...",KR1020100038886A


In [18]:
len(result2)

41951

In [19]:
# 存進MongoDB
# db.rm_patent_us.insert_many(result2.to_dict('records'))

## 切成前後五年存

前五年

US

In [24]:
dateStr = "2010-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2015-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [25]:
db_us_5 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [26]:
# 存進MongoDB
db.rm_patent_us_5.insert_many(db_us_5.to_dict('records'))

後五年

US

In [101]:
dateStr = "2016-01-01T00:00:00.000Z"
start = dateutil.parser.parse(dateStr)

dateEnd = "2020-12-31T00:00:00.000Z"
end = dateutil.parser.parse(dateEnd)

In [ ]:
db_us_10 = pd.DataFrame(db_rm_us.aggregate( [ 
    {"$addFields": { "appDate":
            { "$dateFromString": {
            "dateString": '$appDate',
            "timezone": 'America/New_York'}
            }}
        },
        {"$match": {
            "appDate": { "$gte": start, "$lte": end }}
        }

 ] ))

In [29]:
# 存進MongoDB
db.rm_patent_us_10.insert_many(db_us_10.to_dict('records'))

## 前五年

### DE

In [30]:
data_assignee = db_rm_de_5.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [39]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_de_5.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Robert Bosch GmbH,12,7
1,Robert Bosch GmbH,H04L9,7
2,Robert Bosch GmbH,F01N2610,7
3,Robert Bosch GmbH,G08G1,7
4,Robert Bosch GmbH,00,5


In [40]:
df_cpc.head()

,company,cpc,count
0,Robert Bosch GmbH,12,7
1,Robert Bosch GmbH,H04L9,7
2,Robert Bosch GmbH,F01N2610,7
3,Robert Bosch GmbH,G08G1,7
4,Robert Bosch GmbH,00,5


In [41]:
df_cpc.shape

(646, 3)

In [42]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [43]:
# 轉成company-cpc 的總數表格
table_de = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [44]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,A42,A61,B05,B27,B41,B42,B81,C07,C08,...,G07,G08,G09,H01,H02,H03,H04,H05,Y02,Y10
0,Advantech Service IoT GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,24.0,0.0,0.0,3.0,0.0,0.0,0.0
1,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0
2,Armstrong DLW GmbH,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Audi AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DLOG GES fur ELEKTRONISCHE DATENTECHNIK MBH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,24.0,0.0,0.0,3.0,0.0,0.0,0.0


In [45]:
final_table_de.shape

(41, 29)

In [46]:
# final_table_de.to_csv('rm_company_cpc_de_5.csv',index=0)

### CN

In [55]:
data_assignee = db_rm_cn_5.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [56]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_cn_5.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Jiangsu IoT Research and Development Center,H01L29,156
1,Jiangsu IoT Research and Development Center,H01L21,24
2,Jiangsu IoT Research and Development Center,66325,21
3,Jiangsu IoT Research and Development Center,G06F16,18
4,Jiangsu IoT Research and Development Center,7393,15


In [57]:
df_cpc.head()

,company,cpc,count
0,Jiangsu IoT Research and Development Center,H01L29,156
1,Jiangsu IoT Research and Development Center,H01L21,24
2,Jiangsu IoT Research and Development Center,66325,21
3,Jiangsu IoT Research and Development Center,G06F16,18
4,Jiangsu IoT Research and Development Center,7393,15


In [58]:
df_cpc.shape

(11792, 3)

In [59]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [60]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [61]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A23,A41,A42,A43,A44,A45,A46,A47,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3M innovation intellectual property Co.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3b Technology Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3d-Oxset Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,A Mo Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
final_table_cn.shape

(868, 105)

### US

In [63]:
data_assignee = db_rm_us_5.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [64]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_us_5.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Samsung Electronics Co Ltd,H04W4,258
1,Samsung Electronics Co Ltd,G06F3,232
2,Samsung Electronics Co Ltd,H04W52,119
3,Samsung Electronics Co Ltd,H01L27,116
4,Samsung Electronics Co Ltd,02,103


In [65]:
df_cpc.head()

,company,cpc,count
0,Samsung Electronics Co Ltd,H04W4,258
1,Samsung Electronics Co Ltd,G06F3,232
2,Samsung Electronics Co Ltd,H04W52,119
3,Samsung Electronics Co Ltd,H01L27,116
4,Samsung Electronics Co Ltd,02,103


In [66]:
df_cpc.shape

(20908, 3)

In [67]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [68]:
# 轉成company-cpc 的總數表格
table_us = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [69]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A01,A23,A41,A42,A44,A46,A47,A61,A62,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3d-Oxides,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ABB Schweiz AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,7.0,0.0,0.0,0.0,1.0,1.0,0.0
2,ADVANERGY Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,8.0,0.0,3.0,4.0,0.0
3,AFI Licensing LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
4,AFIRMA CONSULTING & TECHNOLOGIES SL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0


In [70]:
final_table_us.shape

(649, 72)

In [75]:
final_table_us.to_csv('rm_company_cpc_us_5.csv',index=0)

In [72]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_us_5

In [73]:
# title
conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

In [74]:
# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### 合併

In [76]:
# concat de cn us
df_c = pd.concat([final_table_de,final_table_cn], join='outer')
df_c2 = pd.concat([df_c,final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2

cpc_n,company,A01,A23,A41,A42,A43,A44,A45,A46,A47,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,Advantech Service IoT GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,24.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
1,Apple Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0
2,Armstrong DLW GmbH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Audi AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,DLOG GES fur ELEKTRONISCHE DATENTECHNIK MBH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,24.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,ZTE Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,75.0,0.0,1.0,0.0,0.0
645,ZTE TX Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0
646,Zhejiang Shenghui Lighting Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
647,Zhongke Future Of Chip Microelectronics Techno...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2

cpc_n,company,A01,A23,A41,A42,A43,A44,A45,A46,A47,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3M innovation intellectual property Co.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3b Technology Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3d-Oxides,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3d-Oxset Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1466,"Zhuozhou City Heng Qi building materials Co., Ltd",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
1467,Zhuzhou Times Engineering Plastics Technology ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1468,Ztel Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1469,Zyray Wireless Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,2.0,0.0,0.0


In [78]:
df_c2.to_csv('rm_company_cpc_all_5.csv',index=0)

In [79]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_cpc_all_5

In [80]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

In [81]:
# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})

## 後五年

### DE

In [18]:
data_assignee = db_rm_de_10.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [19]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_de_10.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Intel Corp,G06F9,186
1,Intel Corp,G06N3,82
2,Intel Corp,G06F12,53
3,Intel Corp,00,52
4,Intel Corp,G06F3,51


In [20]:
df_cpc.head()

,company,cpc,count
0,Intel Corp,G06F9,186
1,Intel Corp,G06N3,82
2,Intel Corp,G06F12,53
3,Intel Corp,00,52
4,Intel Corp,G06F3,51


In [21]:
df_cpc.shape

(8553, 3)

In [22]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [23]:
# 轉成company-cpc 的總數表格
table_de = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_de = table_de.droplevel(level=0, axis=1)
table_de = table_de.rename(columns={'':'company'})

In [24]:
# 取有英文的部份
c_ls_de = [c for c in table_de.columns if re.search('[A-Za-z]', c)]
final_table_de = table_de[c_ls_de]
final_table_de.head()

cpc_n,company,A01,A24,A41,A42,A44,A45,A47,A61,A62,...,G11,G16,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,ALBA GROUP PLC & Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,AT&S Austria Technologie and Systemtechnik AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0
3,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Accell Group NV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
final_table_de.shape

(328, 74)

In [26]:
# final_table_de.to_csv('rm_company_cpc_de_5.csv',index=0)

### CN

In [27]:
data_assignee = db_rm_cn_10.aggregate( [ 
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}

 ] )


ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])
    
# for data in data_assignee:
#     print(data)

In [28]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_cn_10.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Huawei Technologies Co Ltd,H04W72,1044
1,Huawei Technologies Co Ltd,H04L5,709
2,Huawei Technologies Co Ltd,H04L1,559
3,Huawei Technologies Co Ltd,00,476
4,Huawei Technologies Co Ltd,H04W76,364


In [29]:
df_cpc.head()

,company,cpc,count
0,Huawei Technologies Co Ltd,H04W72,1044
1,Huawei Technologies Co Ltd,H04L5,709
2,Huawei Technologies Co Ltd,H04L1,559
3,Huawei Technologies Co Ltd,00,476
4,Huawei Technologies Co Ltd,H04W76,364


In [30]:
df_cpc.shape

(113707, 3)

In [31]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [32]:
# 轉成company-cpc 的總數表格
table_cn = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_cn = table_cn.droplevel(level=0, axis=1)
table_cn = table_cn.rename(columns={'':'company'})

In [33]:
# 取有英文的部份
c_ls_cn = [c for c in table_cn.columns if re.search('[A-Za-z]', c)]
final_table_cn = table_cn[c_ls_cn]
final_table_cn.head()

cpc_n,company,A01,A21,A23,A24,A41,A42,A43,A44,A45,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,1230 Intelligent Technology Haian Co ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,3M Innovative Properties Co,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
2,"5G IP HOLDINGS, LLC",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
3,63921 Troops of PLA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
4,A Field Of Shenzhen Science And Technology Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [34]:
final_table_cn.shape

(7034, 124)

### US

In [6]:
data_assignee = db_rm_us_10.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}#,
    #{"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    # print(data["_id"])
    ls_company.append(data["_id"].replace("))",")"))
    ls_count.append(data["count"])

In [7]:
ls_c = []
ls_cpc = []
ls_count = []

for c in ls_company:
    data_cpc = db_rm_us_10.aggregate([
        {"$match" : {"current_assignee":{"$regex":c}}},
        {"$unwind" : "$current_assignee"},
        {"$unwind" : "$classification"},
        {"$project" : {"class": {"$split": ["$classification", "/"]}, "qty":1}},
        {"$unwind" : "$class"},
        {"$group" : {"_id" : "$class", "count":{"$sum":1}}},
        {"$sort": {"count":-1}}
        # {"$limit":5}
    ])
    

    for data in data_cpc:
        #print(data)
        ls_c.append(c)
        ls_cpc.append(data["_id"])
        ls_count.append(data["count"])
        
df_cpc = pd.DataFrame((zip(ls_c, ls_cpc, ls_count)), columns = ['company', 'cpc', 'count'])
df_cpc.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,8645
1,Qualcomm Inc,H04W72,6970
2,Qualcomm Inc,H04L1,3851
3,Qualcomm Inc,H04B7,3729
4,Qualcomm Inc,H04W52,1984


In [8]:
df_cpc.head()

,company,cpc,count
0,Qualcomm Inc,H04L5,8645
1,Qualcomm Inc,H04W72,6970
2,Qualcomm Inc,H04L1,3851
3,Qualcomm Inc,H04B7,3729
4,Qualcomm Inc,H04W52,1984


In [9]:
df_cpc.shape

(186283, 3)

In [10]:
# 取cpc前3碼
df_cpc["cpc_n"] = df_cpc.cpc.apply(lambda x: x[0:3])

In [11]:
# 轉成company-cpc 的總數表格
table_us = df_cpc.pivot_table(index='company', values='count', columns='cpc_n',aggfunc=['sum']).fillna(0).reset_index()
table_us = table_us.droplevel(level=0, axis=1)
table_us = table_us.rename(columns={'':'company'})

In [12]:
# 取有英文的部份
c_ls_us = [c for c in table_us.columns if re.search('[A-Za-z]', c)]
final_table_us = table_us[c_ls_us]
final_table_us.head()

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
3,10757616 Canada Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,14.0,0.0,0.0,0.0,1.0,1.0,0.0
4,12a Systems Co Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
final_table_us.shape

(4641, 119)

In [14]:
final_table_us.to_csv('rm_company_cpc_us_10.csv',index=0)

In [15]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection_rm = db.rm_company_cpc_us_10

In [16]:
# title
conn_db_collection_rm.insert_one({"company":final_table_us.columns.tolist()[0],"cpc":final_table_us.columns.tolist()[1:]})

In [17]:
# 內容
for k in final_table_us.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection_rm.insert_one({"company":k[0],"cpc":k[1:]})

### 合併

In [35]:
# concat de cn us
df_c = pd.concat([final_table_de,final_table_cn], join='outer')
df_c2 = pd.concat([df_c,final_table_us], join='outer')
df_c2 = df_c2.fillna(0)
#reset index
t = list(df_c2.sort_index(axis=1).columns)
df_c2 = df_c2.reindex(columns=[t[-1]]+t[:-1])
df_c2

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,ACO Severin Ahlmann GmbH and Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,ALBA GROUP PLC & Co KG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,AT&S Austria Technologie and Systemtechnik AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,102.0,0.0,0.0,0.0
3,AU Optronics Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Accell Group NV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4636,incNETWORKS Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0
4637,mSignia Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
4638,nChain Holdings Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0
4639,u-blox AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,27.0,0.0,1.0,0.0,0.0


In [36]:
# 合併重複的公司
df_c2 = df_c2.groupby(df_c2.company, axis=0).sum().reset_index()
df_c2

cpc_n,company,A01,A21,A22,A23,A24,A41,A42,A43,A44,...,G16,G21,H01,H02,H03,H04,H05,Y02,Y04,Y10
0,#glowontech,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0Chain LLC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,10353744 Canada Ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
3,10757616 Canada Corp,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,14.0,0.0,0.0,0.0,1.0,1.0,0.0
4,1230 Intelligent Technology Haian Co ltd,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11311,incNETWORKS Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,31.0,0.0,0.0,0.0,0.0
11312,mSignia Inc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
11313,nChain Holdings Ltd,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,103.0,0.0,0.0,0.0,0.0
11314,u-blox AG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,40.0,0.0,1.0,0.0,0.0


In [37]:
df_c2.to_csv('rm_company_cpc_all_10.csv',index=0)

In [38]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

conn = pymongo.MongoClient(mongoURI)
db = conn.eva
conn_db_collection = db.rm_company_cpc_all_10

In [39]:
# title
conn_db_collection.insert_one({"company":df_c2.columns.tolist()[0],"cpc":df_c2.columns.tolist()[1:]})

In [40]:
# 內容
for k in df_c2.values.tolist():
    # print({"company":k[0],"cpc":k[1:]})
    conn_db_collection.insert_one({"company":k[0],"cpc":k[1:]})